In [10]:
import pandas as pd
from constants.layoutType import LayoutType
from constants.behaviourType import BehaviourType

In [11]:
# Success rate (fully explore)


def getSuccessRate(df: pd.DataFrame, columnName):
    return len(df[df[columnName] >= 0]) / len(df) * 100


frames = []
for num in [15, 10, 5]:
    for layout in LayoutType.getExperimentLayout():
        dfd = pd.read_csv(f"experimentResults/experiment_{layout.name}_{num}.csv")
        frames.append(dfd)

df_concat = pd.concat(frames)

print(f"Method,50%,75%,90%,100%")
for behaviour in BehaviourType.getExperimentBehaviourType():
    df_behav = df_concat[df_concat["Behaviour"] == behaviour.name]
    successRate50 = getSuccessRate(df_behav, "total50")
    successRate75 = getSuccessRate(df_behav, "total75")
    successRate90 = getSuccessRate(df_behav, "total90")
    successRate100 = getSuccessRate(df_behav, "total100")
    print(
        f"{behaviour.name},{successRate50:.2f}%,{successRate75:.2f}%,{successRate90:.2f}%,{successRate100:.2f}%"
    )

Method,50%,75%,90%,100%
WANDERING,100.00%,97.78%,88.70%,51.85%
GREEDY,100.00%,99.44%,98.15%,88.52%
FRONTIER,100.00%,100.00%,100.00%,98.70%
GREEDY_FRONTIER,100.00%,100.00%,100.00%,97.04%
FRONTIER_ASSIST,100.00%,100.00%,100.00%,98.89%
SCOUT,100.00%,99.81%,99.81%,99.07%
FRONTIER_CENTRAL_GREEDY,100.00%,100.00%,100.00%,91.85%
ZONE_SPLIT,100.00%,100.00%,99.26%,92.22%
ZONE_VORONOI,100.00%,100.00%,100.00%,100.00%


In [12]:
# Success rate (fully explore)


def getSuccessRate(df: pd.DataFrame, columnName):
    return len(df[df[columnName] >= 0]) / len(df) * 100


with open("result_coverage.csv", "w") as f:
    f.write(
        ",".join(
            [
                "num",
                "layout",
                "Method",
                "Coverage 50%",
                "Coverage 75%",
                "Coverage 90%",
                "Coverage 100%",
            ]
        )
    )
    f.write("\n")

    frames = []
    for num in [15, 10, 5]:
        for layout in LayoutType.getExperimentLayout():
            df = pd.read_csv(f"experimentResults/experiment_{layout.name}_{num}.csv")
            frames.append(df)

            df_concat = pd.concat(frames)

            print(f"num,Method,50%,75%,90%,100%")
            for behaviour in BehaviourType.getExperimentBehaviourType():
                df_behav = df_concat[df_concat["Behaviour"] == behaviour.name]
                successRate50 = getSuccessRate(df_behav, "total50")
                successRate75 = getSuccessRate(df_behav, "total75")
                successRate90 = getSuccessRate(df_behav, "total90")
                successRate100 = getSuccessRate(df_behav, "total100")
                print(
                    f"{num},{layout.name},{behaviour.name},{successRate50:.2f}%,{successRate75:.2f}%,{successRate90:.2f}%,{successRate100:.2f}%"
                )
                f.write(f"{num},{layout.name},{behaviour.name},{successRate50:.2f}%,{successRate75:.2f}%,{successRate90:.2f}%,{successRate100:.2f}%")
                f.write("\n")

num,Method,50%,75%,90%,100%
15,PLAIN,WANDERING,100.00%,100.00%,100.00%,100.00%
15,PLAIN,GREEDY,100.00%,100.00%,100.00%,100.00%
15,PLAIN,FRONTIER,100.00%,100.00%,100.00%,100.00%
15,PLAIN,GREEDY_FRONTIER,100.00%,100.00%,100.00%,100.00%
15,PLAIN,FRONTIER_ASSIST,100.00%,100.00%,100.00%,100.00%
15,PLAIN,SCOUT,100.00%,100.00%,100.00%,100.00%
15,PLAIN,FRONTIER_CENTRAL_GREEDY,100.00%,100.00%,100.00%,100.00%
15,PLAIN,ZONE_SPLIT,100.00%,100.00%,100.00%,100.00%
15,PLAIN,ZONE_VORONOI,100.00%,100.00%,100.00%,100.00%
num,Method,50%,75%,90%,100%
15,OBSTACLES,WANDERING,100.00%,100.00%,100.00%,97.50%
15,OBSTACLES,GREEDY,100.00%,100.00%,100.00%,100.00%
15,OBSTACLES,FRONTIER,100.00%,100.00%,100.00%,100.00%
15,OBSTACLES,GREEDY_FRONTIER,100.00%,100.00%,100.00%,100.00%
15,OBSTACLES,FRONTIER_ASSIST,100.00%,100.00%,100.00%,100.00%
15,OBSTACLES,SCOUT,100.00%,100.00%,100.00%,100.00%
15,OBSTACLES,FRONTIER_CENTRAL_GREEDY,100.00%,100.00%,100.00%,100.00%
15,OBSTACLES,ZONE_SPLIT,100.00%,100.00%,100.00%,90.00%
15,OBS

In [35]:
import numpy as np

behaviourNames = [
    behaviour.name for behaviour in BehaviourType.getExperimentBehaviourType()
]

means = {}
stds = {}

with open("result_mean_std.csv", "w") as f:
    f.write(",".join(["num", "layout", "percent"] + behaviourNames))
    f.write("\n")

    for num in [15, 10, 5]:
        for layout in LayoutType.getExperimentLayout():
            df = pd.read_csv(f"experimentResults/experiment_{layout.name}_{num}.csv")
            df = df.replace(-1, np.nan)

            for percent in ["total50", "total75", "total90", "total100"]:
                means[percent] = []
                stds[percent] = []
                for behaviourName in behaviourNames:
                    df_behav = df[df["Behaviour"] == behaviourName]
                    means[percent].append(df_behav[percent].mean())
                    stds[percent].append(df_behav[percent].std())

                meanStds = [
                    meanI + stdI for meanI, stdI in zip(means[percent], stds[percent])
                ]

                row = ",".join(
                    [str(num), str(layout.name), percent]
                    + [
                        f"{meanI:.2f}+{stdI:.2f}"
                        for meanI, stdI in zip(means[percent], stds[percent])
                    ]
                )

                f.write(row)
                f.write("\n")

In [33]:
import plotly.express as px
import numpy as np

behaviourIndependentNames = [
    behaviour.name
    for behaviour in BehaviourType.getExperimentBehaviourType()
    if behaviour.value < 2
]
behaviourCoordinationNames = [
    behaviour.name
    for behaviour in BehaviourType.getExperimentBehaviourType()
    if behaviour.value >= 2
]

behaviourIndependentLabels = ["Wandering", "Greedy"]
behaviourCoordinationLabels = [
    "Distributed Frontier-based",
    "Distributed Frontier-based greedy",
    "Distributed Frontier-based assistance",
    "Role system: Scout",
    "Centralised Frontier-based",
    "Zone-grid split",
    "Zone-Voronoi",
]

labels = [behaviourIndependentLabels, behaviourCoordinationLabels]


def plot(df, strategy, behaviourNames, behaviourLabels):
    fig = px.box(
        df,
        x="Behaviour",
        y="Moves",
        color="Coverage",
        category_orders={
            "Behaviour": behaviourNames,
            "Coverage": ["50%", "75%", "90%", "100%"],
        },
    )

    fig.update_layout(
        title=f"{layout.name} - {num} agents - {strategy}",
        xaxis_title="Methods",
        xaxis=dict(
            tickmode="array",
            tickvals=[*range(len(behaviourNames))],
            ticktext=behaviourLabels,
        ),
    )

    fig.show()


for num in [15]:  # , 10, 5
    for layout in [l for l in LayoutType.getExperimentLayout() if l.value in [1, 2, 9]]:
        df = pd.read_csv(f"experimentResults/experiment_{layout.name}_{num}.csv")
        df = df.replace(-1, np.nan)

        for i, methods in enumerate(
            [behaviourIndependentNames, behaviourCoordinationNames]
        ):
            df_filtered = df[df["Behaviour"].isin(methods)]
            for column in ["total50", "total75", "total90", "total100"]:
                q1 = df_filtered[column].quantile(0.25)
                q3 = df_filtered[column].quantile(0.75)

                df_filtered_clean = df_filtered[
                    (df_filtered[column] > q1) & (df_filtered[column] < q3)
                ]

            df_melted = df_filtered_clean.melt(
                id_vars=["Behaviour", "Round"], var_name="Coverage", value_name="Moves"
            )
            df_melted["Coverage"] = df_melted["Coverage"].map(
                {
                    "total50": "50%", "total75": "75%", "total90": "90%", "total100": "100%",
                }
            )

            plot(df_melted, ["Independent", "Coordination"][i], methods, labels[i])

In [ ]:
import plotly.express as px


def plotSuccessRate(df: pd.DataFrame):
    df_vis = df.copy()
    df_vis["total100"] = df_vis["total100"].apply(
        lambda x: "Success" if x >= 0 else "Unsuccess"
    )

    color_map = {"Success": "royalblue", "Unsuccess": "tomato"}
    fig = px.histogram(
        df_vis,
        x="Behaviour",
        color="total100",
        barmode="group",
        barnorm="percent",
        color_discrete_map=color_map,
        category_orders={
            'Behaviour':[behav.name for behav in BehaviourType.getExperimentBehaviourType()],
            "total100": ["Success", "Unsuccess"]},
    )

    return fig